In [1]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from labelling import pelabelan

# Untuk data bintang 5

In [74]:
url = "https://www.tokopedia.com/vivo/vivo-iqoo-z7x-8-128-80-watt-6000-mah-snapdragon-695-tropical-blue-bd2b1/review"
options=webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver=webdriver.Chrome(options=options)
driver.get(url)

data_scrap=[]
rating_scrap=[]
for i in range(0,1):
    soup=BeautifulSoup(driver.page_source,'lxml')
    time.sleep(2)
    time.sleep(3)
    # driver.find_element(By.CSS_SELECTOR,".checkbox:nth-child(1) [data-testid^='ratingFilter']").click()
    contain_file=soup.find_all('article',attrs={'class':'css-72zbc4'})
    for contain in contain_file:
        review=contain.find('span',attrs={"data-testid":'lblItemUlasan'})
        ratings=contain.find('div',attrs={"data-testid":'icnStarRating'})
        data_scrap.append(review)
        rating_scrap.append(ratings)
    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR,"button[aria-label^='Laman berikutnya']").click()
    time.sleep(3)

In [123]:
data={
    "komentar":data_scrap,
    "rating":rating_scrap
}

In [133]:
data_copy=data.copy()
data_df=pd.DataFrame(data_copy)

In [129]:
data_df.to_csv("test.csv")

In [156]:
df_data=pd.read_csv("test.csv")

In [157]:
# regex 
df_data['rating_komentar']=df_data['rating'].str.get(25)
df_data["komentar_bersih"]=df_data["komentar"].str.replace(r'<[^<>]*>', '', regex=True)

In [160]:
df_data["komentar"]=df_data["komentar_bersih"]
df_data["rating"]=df_data["rating_komentar"]

In [162]:
df_data=df_data[["komentar","rating"]]

In [163]:
df_data["kelas"]=df_data['rating'].apply(pelabelan)

C:\Users\Reza\AppData\Local\Temp/ipykernel_17052/3377772131.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data["kelas"]=df_data['rating'].apply(pelabelan)


In [164]:
df_data

,komentar,rating,kelas
0,"mengeri vivo, thank you produknya...^^good",5,positif
1,"thanks vivi Indonesia, fast respon dan barang ...",5,positif
2,"pesanan di respon hampir 2 hari , lalu kurir t...",2,negatif
3,"Barang Original, Keren, akan tetapi seller lam...",4,positif
4,"Terimakasih vivo dan tokopedia, biarpun banyak...",5,positif
5,"sesuai pesanan, hp nya bagus... ngecas HP sang...",5,positif
6,ORI. puas,5,positif
7,cocok,5,positif
8,"Kualitas barang bagus, sesuai deskripsi",5,positif
9,Barang sudah sampai sesuai pesanan👍👍..fast res...,5,positif


In [167]:
pakaian_negatif = df_data[df_data["kelas"] == "negatif"]

In [168]:
pakaian_negatif

,komentar,rating,kelas
2,"pesanan di respon hampir 2 hari , lalu kurir t...",2,negatif


In [169]:
df_pakaian=pd.read_csv("assets\data_pakaian.csv")

In [170]:
df_pakaian

,Unnamed: 0,komentar,rating
0,0,ukuran m nya terlalu kecil kaya ukuran s,"<div aria-label=""bintang 2"" class=""rating"" dat..."
1,1,"bahannya kurang bisa menyerap keringat, dan wa...","<div aria-label=""bintang 5"" class=""rating"" dat..."
2,2,kaos tidak sesuai size nya dan minta 3 warna c...,"<div aria-label=""bintang 1"" class=""rating"" dat..."
3,3,bahannya enak adem. sablon keliatannya ga akan...,"<div aria-label=""bintang 5"" class=""rating"" dat..."
4,4,keren,"<div aria-label=""bintang 5"" class=""rating"" dat..."
5,5,Bahannya tipis. banget kelihatan nerawang....,"<div aria-label=""bintang 3"" class=""rating"" dat..."
6,6,bahan tipis tp sesuai lah dengan harga,"<div aria-label=""bintang 4"" class=""rating"" dat..."
7,7,"kualitas kain standar, cenderung tipis.\nsesua...","<div aria-label=""bintang 4"" class=""rating"" dat..."
8,8,,"<div aria-label=""bintang 3"" class=""rating"" dat..."
9,9,"pesanan sudah diterima, produk bagus, sesuai d...","<div aria-label=""bintang 5"" class=""rating"" dat..."


In [171]:
df_pakaian['rating']=df_pakaian['rating'].str.get(25)

In [174]:
df_pakaian["kelas"]=df_pakaian["rating"].apply(pelabelan)

In [175]:
pakaian_negatif = df_pakaian[df_pakaian["kelas"] == "negatif"]

In [186]:
"".join(map(str,pakaian_negatif["komentar"].to_list()))

'ukuran m nya terlalu kecil kaya ukuran skaos tidak sesuai size nya dan minta 3 warna cm dikasih 1 warna'

In [181]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [187]:
 wordcloud = WordCloud().generate("".join(map(str,pakaian_negatif["komentar"].to_list())))
    
fig, ax = plt.subplots()
ax.imshow(wordcloud, interpolation='bilinear')
ax.axis("off")
plt.show()

ValueError: Only supported for TrueType fonts